In [0]:
from pyspark.sql.functions import col, datediff, current_date, count

# ✅ Load the cleaned Silver table
silver_df = spark.table("silver_db.jobs_silver")
silver_df.show(5)




+-------+--------------------+-----------+-----------------+----------+--------------------+--------+------------+------------------+--------------------+-------------------+----------+------------+-----------------+--------+
| job_id|           title_raw|    company|     location_raw| post_date|                 url|type_raw|  levels_raw|    categories_raw|            contents|         _ingest_ts|_run_label|     _source|         location|    type|
+-------+--------------------+-----------+-----------------+----------+--------------------+--------+------------+------------------+--------------------+-------------------+----------+------------+-----------------+--------+
|3104241|Merchandise and S...|    Walmart|  Albuquerque, NM|2025-08-13|https://www.themu...|external|   Mid Level|Data and Analytics|Position Summary....|2025-08-27T23:02:58|    manual|the_muse_api|  Albuquerque, NM|external|
|4278354|Senior Data Scien...|   Bluecore|     New York, NY|2024-11-01|https://www.themu...|exte

In [0]:
from pyspark.sql.functions import col, current_date, datediff, to_date

# ➡️ Transform Silver to Gold
gold_df = (
    silver_df
    .select(
        "job_id", "company", "title_raw", "location", "type",
        "post_date", "contents", "categories_raw", "levels_raw", "url"
    )
    .withColumn("post_date_cleaned", to_date(col("post_date"), "yyyy-MM-dd"))
    .withColumn("job_age", datediff(current_date(), col("post_date_cleaned")))
    .filter(col("post_date_cleaned").isNotNull())
    .filter(col("job_age") <= 60)  # Filter: last 60 days
)



In [0]:
# ✅ Save to Delta table in gold_db
spark.sql("CREATE DATABASE IF NOT EXISTS gold_db")

gold_table_name = "jobs_gold"
full_gold_path = f"gold_db.{gold_table_name}"

gold_df.write \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .format("delta") \
    .saveAsTable(full_gold_path)


In [0]:
# 📊 Show 10 rows
display(spark.sql(f"SELECT * FROM {full_gold_path} LIMIT 10"))

# 📈 Aggregations
print("Jobs by type:")
gold_df.groupBy("type").agg(count("*").alias("job_count")).orderBy(col("job_count").desc()).show()

print("Jobs by location:")
gold_df.groupBy("location").agg(count("*").alias("job_count")).orderBy(col("job_count").desc()).show(10)

print("Jobs by company:")
gold_df.groupBy("company").agg(count("*").alias("job_count")).orderBy(col("job_count").desc()).show(10)


job_id company title location type post_date_cleaned contents categories_raw levels_raw url job_age title_raw post_date 3104241 Walmart null Albuquerque, NM external 2025-08-13 Position Summary...Do you like to work on your feet and keep things neat and organized? Our merchandising & stocking associates connect all of the dots to make sure members can find everything they have on their shopping list. From every day needs to special occasions, members need you, as their mini tour guide, to take them that special product. Depending on the shift you work, your job could include moving inventory in the backroom, unloading trucks, fulfilling club pick-up orders or helping members while stocking shelves. From unloading trucks in the summer to filling ice cream in the freezer, this fast-paced job can be physically demanding - it's like being paid to go to the gym! Be a part of a great team with a common goal - making sure members can find more of what they love, for less. You will sweep us off our feet if:• You thrive in fast-paced environments • You keep member satisfaction as your top priority• You're comfortable with change and quickly adapt to different work scenarios• You're a problem solver who tackles obstacles head-on to ensure each task is completed with excellence• You are able to pick up boxes and other heavy objects weighing more than 25 poundsYou will make an impact by:• Promptly unloading trucks • Assisting fellow associates as needed throughout the store• Sorting and stocking products on shelves and in the backroom• Engaging with vendors and drivers with a positive attitude• Maintaining a clean, neat, and member-ready areaThe merchandising & stocking associate role is a great way to start a fulfilling career at Sam's Club. Apply now!The above information has been designed to indicate the general nature and level of work performed in the role. It is not designed to contain or be interpreted as a comprehensive inventory of all duties, responsibilities and qualifications required of employees assigned to this job. The full job description can be made available as part of the hiring process.What you'll do...Provides member service by acknowledging the member and identifying member needs; providing guidance and support to members regarding self- service technology; assisting members with purchasing decisions; locating merchandise; resolving member issues and concerns; and promoting the company's products and services.Maintains safety of facility according to company policies and procedures by conducting safety sweeps; following forklift spotting procedures; following procedures for handling and disposing of hazardous materials; following company steel standard guidelines; and correcting and reporting unsafe situations to management.Maintains the sales floor and merchandise presentation in accordance with company policies and procedures by properly zoning the area; stocking, arranging and organizing merchandise; setting up, cleaning, and organizing product displays; removing damaged goods; signing and pricing merchandise according to company policies and procedures; identifying shrink and damages; and securing fragile and high-shrink merchandise.Monitors food and merchandise quality by ensuring product rotation, code dating, product recalls, and sanitation standards are followed according to company policies and procedures; and ensuring prominent display of promotional and seasonal merchandise.Complies with company policies, procedures, and standards of ethics and integrity by implementing related action plans; using the Open Door Policy; and applying these in executing business processes and practices.Completes work assignments and priorities by using policies, data, and resources; collaborating with managers, co-workers, customers, and other business partners; identifying priorities, deadlines, and expectations; carrying out tasks; communicating progress and information; determining and recommending ways to address improve

Jobs by type:
+--------+---------+
|    type|job_count|
+--------+---------+
|external|      441|
+--------+---------+

Jobs by location:
+-----------------+---------+
|         location|job_count|
+-----------------+---------+
|     San Jose, CA|       40|
|      Seattle, WA|       23|
|     New York, NY|       22|
|        Singapore|       16|
| Bangalore, India|       15|
|  Los Angeles, CA|       13|
|   Menlo Park, CA|        8|
|San Francisco, CA|        8|
| Hyderabad, India|        8|
|Flexible / Remote|        8|
+-----------------+---------+
only showing top 10 rows
Jobs by company:
+--------------+---------+
|       company|job_count|
+--------------+---------+
|        TikTok|       95|
|          Meta|       65|
|    CVS Health|       50|
|JPMorgan Chase|       22|
|           IBM|       19|
|    GE Vernova|       12|
|    Mastercard|       12|
|       Kyndryl|       10|
|   Capital One|       10|
|  The Hartford|        8|
+--------------+---------+
only showing top 10 ro

In [0]:
pandas_df = gold_df.limit(1000).toPandas()  # Adjust limit if needed





In [0]:
import base64
from IPython.display import HTML

csv = pandas_df.to_csv(index=False)
b64 = base64.b64encode(csv.encode()).decode()

file_name = "jobs_gold.csv"

html = f'<a download="{file_name}" href="data:text/csv;base64,{b64}" target="_blank">📥 Click here to download CSV</a>'
display(HTML(html))


